# Description

The notebook automates the process of pushing a pair of scenarios (baseline and alternative) through the simulation and optimization pipeline, comparing outputs for each scenario. Key outputs include harvested area (hectares), harvested volume (m³), and growth (m³). The notebook also reports stand age class distribution, old growth attributes, and tree species diversity at the beginning and end of the planning horizon for each scenario. Additionally, it calculates biodiversity and socioeconomic indicators, while quantifying differences between baseline and alternative scenarios in terms of carbon stocks and net emissions over a customizable simulation horizon, set to 100 years by default (the system can easily be parametrized to simulate shorter or longer horizons).

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import time
import pickle
import os
import geopandas as gpd
import ws3.forest, ws3.core
import csv
from functools import partial, wraps
import distance
import operator
import shutil
from IPython.display import display
import libcbm
from util import compile_events, cbm_report, compile_scenario_maxstock, plot_scenario_maxstock, run_cbm_emissionstock, run_scenario, plugin_c_curves, plugin_c_curves, cbm_report_both, compare_ws3_cbm, compare_ws3_cbm_both, track_system_stock, track_system_emission, compile_scenario_minemission, plot_scenario_minemission, kpi_age, kpi_species, cmp_c_ss, cmp_c_se, results_scenarios, bootstrap_ogi, compare_kpi_age, epsilon_computer, tradeoff_biodiversity_cs, tradeoff_hv_cs, tradeoff_hv_biodiversity, inventory_processing, curve_points_generator, fm_bootstrapper, carbon_curve_points

In [2]:
# Input paths
shapefile_path = './data/shp_files/tsa17.shp/stands selection.shp'
# shapefile_path = './data/shp_files/tsa01.shp/stands.shp'

In [3]:
stands_org = gpd.read_file(shapefile_path)
stands_org = stands_org

In [4]:
# Initialize the input parameters
base_year = 2020
horizon = 10
period_length = 10
max_age = 1000
n_steps = 100
tvy_name = 'totvol'
max_harvest = 1.0
case_study = 'ecotrust'
scenario_names = ['lowest carbon stock', 'business as usual', '40% of highest carbon stock', '60% of highest carbon stock', '20% of highest carbon stock', 'highest carbon stock']
obj_mode = 'max_hv'
hwp_pool_effect_value = 0
release_immediately_value = 0
displacement_effect = 0
clt_percentage = 0
credibility = 0
budget_input = 10000000

In [5]:
# Reading yld table and changing AU column type into integer
yld = pd.read_csv('./data/yld.csv')
yld['AU'] = yld['AU'].astype(int)


# Getting names and codes of canfi_species. This table will be used to add canfi_species column into yldmerged
canf = pd.read_csv('data/canfi_species_revised.csv')
canf = canf[['name','canfi_species']].set_index('name')

In [6]:
stands = inventory_processing(stands_org, canf)

In [7]:
curve_points_table = curve_points_generator(stands, yld, canf)

In [8]:
fm = fm_bootstrapper(base_year, horizon, period_length, max_age, stands, curve_points_table, tvy_name)

In [9]:
c_curves_p, c_curves_f = carbon_curve_points(fm)

/home/salar2/project/ecotrust-dss-/venv-3.10/lib/python3.10/site-packages/libcbm/resources/__init__.py:112: RuntimeWarning: untested linux distribution: Linux-6.8.0-48-generic-x86_64-with-glibc2.39
  warnings.warn(message, RuntimeWarning)
/home/salar2/project/ecotrust-dss-/venv-3.10/lib/python3.10/site-packages/libcbm/resources/__init__.py:112: RuntimeWarning: untested linux distribution: Linux-6.8.0-48-generic-x86_64-with-glibc2.39
  warnings.warn(message, RuntimeWarning)


In [10]:
# Plug carbon and old groth yield curves into forest model
plugin_c_curves(fm, c_curves_p, c_curves_f)
bootstrap_ogi(fm)

In [11]:
Start = time.time()

epsilon, cs_max = epsilon_computer(fm, clt_percentage, hwp_pool_effect_value, displacement_effect, release_immediately_value, n=10, solver=ws3.opt.SOLVER_PULP)
# scenario_names = scenario_options.get(case_study, [])
for scenario_name in scenario_names:
    print(f"Running for {case_study}_{obj_mode}_{scenario_name}...")
    results_scenarios(fm, 
                      clt_percentage, 
                      credibility, 
                      budget_input, 
                      n_steps, 
                      max_harvest, 
                      scenario_name, 
                      displacement_effect, 
                      hwp_pool_effect_value, 
                      release_immediately_value, 
                      case_study, 
                      obj_mode, 
                      epsilon,
                      cs_max,
                      pickle_output_base=False, 
                      pickle_output_alter=False)
print('It took', round((time.time() - Start) / 60, 1), "minutes to run all scenarios.")

It took 0.1 minutes to run all scenarios.


In [12]:
tradeoff_biodiversity_cs(fm, clt_percentage, hwp_pool_effect_value, displacement_effect, release_immediately_value, n=4, solver=ws3.opt.SOLVER_PULP)

[np.float64(190025.79134199675), np.float64(197087.96564232087), np.float64(203967.8645683486), np.float64(210302.99454379527)]
[np.float64(6095462.262005491), np.float64(6306940.99501847), np.float64(6518419.730516184), np.float64(6729898.464373719)]


([np.float64(190025.79134199675),
  np.float64(197087.96564232087),
  np.float64(203967.8645683486),
  np.float64(210302.99454379527)],
 [np.float64(6095462.262005491),
  np.float64(6306940.99501847),
  np.float64(6518419.730516184),
  np.float64(6729898.464373719)])

In [13]:
epsilon, cs_max = epsilon_computer(fm, clt_percentage, hwp_pool_effect_value, displacement_effect, release_immediately_value, n=4, solver=ws3.opt.SOLVER_PULP)
tradeoff_hv_cs(fm, clt_percentage, hwp_pool_effect_value, displacement_effect, release_immediately_value, epsilon, cs_max, n=4, solver=ws3.opt.SOLVER_PULP)

[np.float64(5.795639600191193e-07), np.float64(670240.3602947406), np.float64(1276613.556441729), np.float64(1819310.4021455366), np.float64(2039009.5075062783)]
[np.float64(6941377.198759295), np.float64(6729898.464549068), np.float64(6518419.730403796), np.float64(6306940.996022622), np.float64(6095462.262168121)]


([np.float64(5.795639600191193e-07),
  np.float64(670240.3602947406),
  np.float64(1276613.556441729),
  np.float64(1819310.4021455366),
  np.float64(2039009.5075062783)],
 [np.float64(6941377.198759295),
  np.float64(6729898.464549068),
  np.float64(6518419.730403796),
  np.float64(6306940.996022622),
  np.float64(6095462.262168121)])

In [14]:
tradeoff_hv_biodiversity(fm, clt_percentage, hwp_pool_effect_value, displacement_effect, release_immediately_value, n=4, solver=ws3.opt.SOLVER_PULP)

[np.float64(-6.4246532164073265e-06), np.float64(759869.0174471902), np.float64(1411898.0307286978), np.float64(1964821.4737019618)]
[np.float64(215092.7638864148), np.float64(206564.68427197606), np.float64(198036.60462904882), np.float64(189508.524994619)]


([np.float64(-6.4246532164073265e-06),
  np.float64(759869.0174471902),
  np.float64(1411898.0307286978),
  np.float64(1964821.4737019618)],
 [np.float64(215092.7638864148),
  np.float64(206564.68427197606),
  np.float64(198036.60462904882),
  np.float64(189508.524994619)])